In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold

In [2]:
# Get the current working directory
current_dir = os.getcwd()
print("Current Working Directory:", current_dir)

Current Working Directory: /Users/cyuab/Desktop/time-series-classification-cleavage/code/sota


In [3]:
# Load
df = pd.read_pickle("../../data/dataset_preprocessed.pkl")

# Prepare Dicleave dataset

In [4]:
def extract_dicleave_patterns(row, pattern_st):
    length = 14
    df_temp = df.copy()
    cleavage_window = row.sequence[(start:=pattern_st):(end:=start + length)]
    window_dot_bracket = row.dot_bracket[start:end]
    cleavage_window_comp = row.Seq_Compl[start:end]
    return cleavage_window, window_dot_bracket, cleavage_window_comp

In [5]:
# 5p
before_center = 7
away_cleavage = 6
df_temp = df
df_temp = df_temp.rename(columns={'Name': 'name', 'Seq': 'sequence', 'SS': 'dot_bracket'})
df_temp2 = df_temp.copy()
df_temp[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_1_End-before_center), axis=1, result_type="expand")
# https://stackoverflow.com/questions/29517072/add-column-to-dataframe-with-constant-value
df_temp['label'] = '1'
df_temp2[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp2.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_1_End-before_center-away_cleavage), axis=1, result_type="expand")
df_temp2['label'] = '0'
df_res = pd.concat([df_temp, df_temp2], ignore_index=True)
df_res = df_res[["name", "sequence", "dot_bracket", "cleavage_window", "window_dot_bracket", "cleavage_window_comp", "label"]]
df_res['cleavage_window_comp'] = df_res['cleavage_window_comp'].str.replace('_','O')


In [6]:
df_res

,name,sequence,dot_bracket,cleavage_window,window_dot_bracket,cleavage_window_comp,label
0,hsa-let-7a-1,UGGGAUGAGGUAGUAGGUUGUAUAGUUUUAGGGUCACACCCACCAC...,(((((.(((((((((((((((((((((.....(((...((((.......,UAUAGUUUUAGGGU,(((((((.....((,AUAUCAAOOOOOUA,1
1,hsa-let-7a-2,AGGUUGAGGUAGUAGGUUGUAUAGUUUAGAAUUACAUCAAGGGAGA...,(((..(((.(((.(((((((((((((.........(((......))...,UAUAGUUUAGAAUU,(((((((.......,AUGUCAAOOOOOOO,1
2,hsa-let-7a-3,GGGUGAGGUAGUAGGUUGUAUAGUUUGGGGCUCUGCCCUGCUAUGG...,(((.(((((((((((((((((((((((((((...)))))).........,UAUAGUUUGGGGCU,(((((((((((((.,AUAUCAAGUCCCGO,1
3,hsa-let-7b,CGGGGUGAGGUAGUAGGUUGUGUGGUUUCAGGGCAGUGAUGUUGCC...,(((((.(((((((((((((((((((((((.((((((.....)))))...,UGUGGUUUCAGGGC,(((((((((.((((,AUAUCAAGGOCCCG,1
4,hsa-let-7c,GCAUCCGGGUUGAGGUAGUAGGUUGUAUGGUUUAGAGUUACACCCU...,((.((((((..(((.(((.(((((((((((((..((.(..((...)...,UAUGGUUUAGAGUU,(((((((..((.(.,AUGUCAAOOUUOAO,1
...,...,...,...,...,...,...,...
1649,hsa-mir-10399,AAUUACAGAUUGUCUCAGAGAAAACAAAUGAGUUACUCUCUCGGAC...,..((((((.((((((.(((((.(((......)))..))))).))))...,AUUGUCUCAGAGAA,.((((((.(((((.,OAACAGGOUCUCUO,0
1650,hsa-mir-10400,CGGCGGCGGCGGCUCUGGGCGAGGCGGCGGGGCCUGGGCUCCCGGA...,.(((..((.((.(((.....))).)).))..)))......((((.....,GCGGCUCUGGGCGA,.((.(((.....)),OGCOGAGOOOOOCU,0
1651,hsa-mir-10401,CGUGUGGGAAGGCGUGGGGUGCGGACCCCGGCCCGACCUCGCCGUC...,((.((((((.(((((.((((.(((...))))))).))...))).))...,GAAGGCGUGGGGUG,((.(((((.((((.,CUOCCGCAOCCCGO,0
1652,hsa-mir-10396b,CGGCGGGGCUCGGAGCCGGGCUUCGGCCGGGCCCCGGGCCCUCGAC...,(((((((((((((.(((.(((....))).))).)))))))).)).)...,GCUCGGAGCCGGGC,((((((.(((.(((,CGGGCCOCGGOCCG,0


In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(skf.split(df_res, df_res['label'])):
    df_train = df_res.iloc[train_idx]
    df_test = df_res.iloc[test_idx]
    os.makedirs(os.path.dirname(path := f"Dicleave-data/training_5p_{fold}.csv"), exist_ok=True); df_train.to_csv(path, index=True)
    os.makedirs(os.path.dirname(path := f"Dicleave-data/test_5p_{fold}.csv"), exist_ok=True); df_test.to_csv(path, index=True)

In [ ]:
# 3p
before_center = 7
away_cleavage = 6
df_temp = df
df_temp = df_temp.rename(columns={'Name': 'name', 'Seq': 'sequence', 'SS': 'dot_bracket'})
df_temp2 = df_temp.copy()
df_temp[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_2_Start-before_center), axis=1, result_type="expand")
# https://stackoverflow.com/questions/29517072/add-column-to-dataframe-with-constant-value
df_temp['label'] = '1'
df_temp2[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp2.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_2_Start-before_center+away_cleavage), axis=1, result_type="expand")
df_temp2['label'] = '0'
df_res = pd.concat([df_temp, df_temp2], ignore_index=True)
df_res = df_res[["name", "sequence", "dot_bracket", "cleavage_window", "window_dot_bracket", "cleavage_window_comp", "label"]]
df_res['cleavage_window_comp'] = df_res['cleavage_window_comp'].str.replace('_','O')


In [10]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(skf.split(df_res, df_res['label'])):
    df_train = df_res.iloc[train_idx]
    df_test = df_res.iloc[test_idx]
    os.makedirs(os.path.dirname(path := f"Dicleave-data/training_3p_{fold}.csv"), exist_ok=True); df_train.to_csv(path, index=True)
    os.makedirs(os.path.dirname(path := f"Dicleave-data/test_3p_{fold}.csv"), exist_ok=True); df_test.to_csv(path, index=True)

In [ ]:
# from sklearn.model_selection import train_test_split, cross_val_score
# df_res_train, df_res_test = train_test_split(df_res, test_size=0.2, random_state=18)
# df_res_train['cleavage_window_comp'] = df_res_train['cleavage_window_comp'].str.replace('_','O')
# df_res_test['cleavage_window_comp'] = df_res_test['cleavage_window_comp'].str.replace('_','O')
# # df_res_train.to_csv("../sota/DiCleave/training_5p.csv", index=True)
# # df_res_test.to_csv("../sota/DiCleave/test_5p.csv", index=True)
# os.makedirs(os.path.dirname(path := "../data/DiCleave/training_5p.csv"), exist_ok=True); df_res_train.to_csv(path, index=True)
# os.makedirs(os.path.dirname(path := "../data/DiCleave/test_5p.csv"), exist_ok=True); df_res_test.to_csv(path, index=True)
# # os.makedirs(os.path.dirname(path := "../data/DiCleave/training_5p.csv"), exist_ok=True); df_res_train.to_csv(path, index=True)
# # os.makedirs(os.path.dirname(path := "../data/DiCleave/test_5p.csv"), exist_ok=True); df_res_test.to_csv(path, index=True)

In [11]:
# Multi
# 5p
before_center = 7
away_cleavage = 6
df_temp = df
df_temp = df_temp.rename(columns={'Name': 'name', 'Seq': 'sequence', 'SS': 'dot_bracket'})
df_temp2 = df_temp.copy()
df_temp[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_1_End-before_center), axis=1, result_type="expand")
# https://stackoverflow.com/questions/29517072/add-column-to-dataframe-with-constant-value
df_temp['label'] = '1'
df_temp2[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp2.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_1_End-before_center-away_cleavage), axis=1, result_type="expand")
df_temp2['label'] = '0'
df_res_5p = pd.concat([df_temp, df_temp2], ignore_index=True)
# 3p
before_center = 7
away_cleavage = 6
df_temp = df
df_temp = df_temp.rename(columns={'Name': 'name', 'Seq': 'sequence', 'SS': 'dot_bracket'})
df_temp2 = df_temp.copy()
df_temp[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_2_Start-before_center), axis=1, result_type="expand")
# https://stackoverflow.com/questions/29517072/add-column-to-dataframe-with-constant-value
df_temp['label'] = '2'
df_temp2[["cleavage_window", "window_dot_bracket", "cleavage_window_comp"]] = df_temp2.apply(
        lambda row: extract_dicleave_patterns(row, row.miRNA_2_Start-before_center+away_cleavage), axis=1, result_type="expand")
df_temp2['label'] = '0'
df_res_3p = pd.concat([df_temp, df_temp2], ignore_index=True)
df_res = pd.concat([df_res_3p, df_res_5p], ignore_index=True)
df_res = df_res[["name", "sequence", "dot_bracket", "cleavage_window", "window_dot_bracket", "cleavage_window_comp", "label"]]
df_res['cleavage_window_comp'] = df_res['cleavage_window_comp'].str.replace('_','O')


In [12]:
df_res

,name,sequence,dot_bracket,cleavage_window,window_dot_bracket,cleavage_window_comp,label
0,hsa-let-7a-1,UGGGAUGAGGUAGUAGGUUGUAUAGUUUUAGGGUCACACCCACCAC...,(((((.(((((((((((((((((((((.....(((...((((.......,GAGAUAACUAUACA,).)))))))))))),COCUGUUGAUAUGU,2
1,hsa-let-7a-2,AGGUUGAGGUAGUAGGUUGUAUAGUUUAGAAUUACAUCAAGGGAGA...,(((..(((.(((.(((((((((((((.........(((......))...,GAGAUAACUGUACA,..)))))))))))),OOCUAUUGAUAUGU,2
2,hsa-let-7a-3,GGGUGAGGUAGUAGGUUGUAUAGUUUGGGGCUCUGCCCUGCUAUGG...,(((.(((((((((((((((((((((((((((...)))))).........,GGGAUAACUAUACA,.....))))))))),OOOOOUUGAUAUGU,2
3,hsa-let-7b,CGGGGUGAGGUAGUAGGUUGUGUGGUUUCAGGGCAGUGAUGUUGCC...,(((((.(((((((((((((((((((((((.((((((.....)))))...,AAGAUAACUAUACA,).....)))))))),UOOOOOUGGUGUGU,2
4,hsa-let-7c,GCAUCCGGGUUGAGGUAGUAGGUUGUAUGGUUUAGAGUUACACCCU...,((.((((((..(((.(((.(((((((((((((..((.(..((...)...,GAGUUAACUGUACA,.).))))))))))),OUOAGUUGGUAUGU,2
...,...,...,...,...,...,...,...
3303,hsa-mir-10399,AAUUACAGAUUGUCUCAGAGAAAACAAAUGAGUUACUCUCUCGGAC...,..((((((.((((((.(((((.(((......)))..))))).))))...,AUUGUCUCAGAGAA,.((((((.(((((.,OAACAGGOUCUCUO,0
3304,hsa-mir-10400,CGGCGGCGGCGGCUCUGGGCGAGGCGGCGGGGCCUGGGCUCCCGGA...,.(((..((.((.(((.....))).)).))..)))......((((.....,GCGGCUCUGGGCGA,.((.(((.....)),OGCOGAGOOOOOCU,0
3305,hsa-mir-10401,CGUGUGGGAAGGCGUGGGGUGCGGACCCCGGCCCGACCUCGCCGUC...,((.((((((.(((((.((((.(((...))))))).))...))).))...,GAAGGCGUGGGGUG,((.(((((.((((.,CUOCCGCAOCCCGO,0
3306,hsa-mir-10396b,CGGCGGGGCUCGGAGCCGGGCUUCGGCCGGGCCCCGGGCCCUCGAC...,(((((((((((((.(((.(((....))).))).)))))))).)).)...,GCUCGGAGCCGGGC,((((((.(((.(((,CGGGCCOCGGOCCG,0


In [13]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(skf.split(df_res, df_res['label'])):
    df_train = df_res.iloc[train_idx]
    df_test = df_res.iloc[test_idx]
    os.makedirs(os.path.dirname(path := f"Dicleave-data/training_multi_{fold}.csv"), exist_ok=True); df_train.to_csv(path, index=True)
    os.makedirs(os.path.dirname(path := f"Dicleave-data/test_multi_{fold}.csv"), exist_ok=True); df_test.to_csv(path, index=True)

# Evaluate result

In [18]:
# 3p and 5p

In [19]:
# Apply this for the 5 set, each for 3p and 5p
df_test = pd.read_csv('DiCleave-data/test_3p_4.csv')
y_true = df_test[['label']].copy()
y_true
y_pred = pd.read_csv('DiCleave-data/training_3p_4_model/model_1_result.txt', skiprows=2, header=None)
y_pred.columns = ['label']
y_pred

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
f1_score = 2 * tp / (2 * tp + fp + fn)
mcc = matthews_corrcoef(y_true, y_pred)
print(accuracy, specificity, sensitivity, f1_score, mcc)
# Result
# 5p_0: 0.8277945619335347 0.7575757575757576 0.8975903614457831 0.8394366197183099 0.6618695056324055
# 5p_1: 0.8429003021148036 0.7515151515151515 0.9337349397590361 0.856353591160221 0.697188473308243
# 5p_2: 0.7764350453172205 0.5963855421686747 0.9575757575757575 0.8102564102564103 0.5936510632926744
# 5p_3: 0.8217522658610272 0.927710843373494 0.7151515151515152 0.8 0.6581794350553273
# 5p_4: 0.8212121212121212 0.9151515151515152 0.7272727272727273 0.802675585284281 0.6540718079401727

# 3p_0: 0.8580060422960725 0.8484848484848485 0.8674698795180723 0.8597014925373134 0.7161116152027885
# 3p_1: 0.8489425981873112 0.8545454545454545 0.8433734939759037 0.8484848484848485 0.6979444307109178
# 3p_2: 0.8851963746223565 0.8734939759036144 0.896969696969697 0.8862275449101796 0.7706325046213385
# 3p_3: 0.8126888217522659 0.9457831325301205 0.6787878787878788 0.7832167832167832 0.6484485250159162
# 3p_4: 0.8666666666666667 0.9333333333333333 0.8 0.8571428571428571 0.7399400733959437

0.8666666666666667 0.9333333333333333 0.8 0.8571428571428571 0.7399400733959437


In [32]:
 from sklearn.metrics import (
        accuracy_score, recall_score, f1_score,
        matthews_corrcoef, confusion_matrix
    )

In [38]:
def multiclass_specificity(cm):
    FP = cm.sum(axis=0) - np.diag(cm)
    TN = cm.sum() - (FP + cm.sum(axis=1) - np.diag(cm) + np.diag(cm))
    return np.mean(TN / (TN + FP))

In [48]:
# Mutli
df_test = pd.read_csv('DiCleave-data/test_multi_4.csv')
y_true = df_test[['label']].copy()
y_true
y_pred = pd.read_csv('DiCleave-data/training_multi_4_model/model_1_result.txt', skiprows=2, header=None)
y_pred.columns = ['label']
y_pred
accuracy = accuracy_score(y_true, y_pred)

sensitivity = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")
mcc = matthews_corrcoef(y_true, y_pred)

cm = confusion_matrix(y_true, y_pred)
specificity = multiclass_specificity(cm)
print(accuracy, specificity, sensitivity, f1, mcc)
# Result
# multi_0: 0.8806646525679759 0.9378945316965628 0.8892665967357481 0.8800017154773756 0.8124776801389478
# multi_1: 0.8761329305135952 0.9263019821220365 0.8602896434221735 0.8733666879462872 0.8007713097418911
# multi_2: 0.8640483383685801 0.9284866117734246 0.8740999335619509 0.8649353706557097 0.7867447278557491
# multi_3: 0.6641452344931922 0.787589605734767 0.5849491897830267 0.5191238763345025 0.4660830645093024
# multi_4: 0.8199697428139183 0.894672531769306 0.8041746772864599 0.8152999994388557 0.7096916354214782

0.8199697428139183 0.894672531769306 0.8041746772864599 0.8152999994388557 0.7096916354214782
